In [41]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from src.data_quality_checks import check_missing_data
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import shap

# Data Preparation

Load and prepare the data

In [39]:
df = pd.read_csv('../data/MachineLearningRating_v3.txt', sep='|', low_memory=False)

# df = pd.read_csv('../data/cleaned_data.csv', header=None,low_memory=False)

In [40]:
# selecting relevant columns
relevant_cols = ['TotalPremium', 'TotalClaims', 'Gender', 'PostalCode', 'Province', 'VehicleType', 'RegistrationYear', 'NewVehicle']

df_relevant_cols = df[relevant_cols]

In [42]:
# checking for columns with missing values
missing_data = check_missing_data(df_relevant_cols)
print(missing_data)

   Column Name  Missing Values  Percentage Missing
2       Gender            9536            0.953507
5  VehicleType             552            0.055195
7   NewVehicle          153295           15.327998


In [43]:
# data cleaning
df_relevant_cols.loc[:, 'VehicleType'] = df_relevant_cols['VehicleType'].fillna(df_relevant_cols['VehicleType'].mode()[0])
df_relevant_cols.loc[:, 'NewVehicle'] = df_relevant_cols['NewVehicle'].fillna(df_relevant_cols['NewVehicle'].mode()[0])
df_relevant_cols.loc[:, 'Gender'] = df_relevant_cols['Gender'].fillna(df_relevant_cols['Gender'].mode()[0])

In [44]:
# checking for columns with missing values
missing_data = check_missing_data(df_relevant_cols)
print(missing_data)

Success: No missing values.


In [45]:
df_relevant_cols.head(10)

,TotalPremium,TotalClaims,Gender,PostalCode,Province,VehicleType,RegistrationYear,NewVehicle
0,21.929825,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months
1,21.929825,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months
2,0.000000,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months
3,512.848070,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months
4,0.000000,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months
5,3.256435,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months
6,50.474737,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months
7,35.332316,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months
8,0.000000,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months
9,1.009474,0.0,Not specified,1459,Gauteng,Passenger Vehicle,2004,More than 6 months


Encoding categorical data

In [46]:

categorical_features = ['Province', 'PostalCode', 'Gender', 'VehicleType', 'RegistrationYear', 'NewVehicle']
encoder = LabelEncoder()

for feature in categorical_features:
    df_relevant_cols[feature] = encoder.fit_transform(df_relevant_cols[feature])

C:\Users\Firew Ayele\AppData\Local\Temp\ipykernel_18500\1617156014.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant_cols[feature] = encoder.fit_transform(df_relevant_cols[feature])
C:\Users\Firew Ayele\AppData\Local\Temp\ipykernel_18500\1617156014.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant_cols[feature] = encoder.fit_transform(df_relevant_cols[feature])
C:\Users\Firew Ayele\AppData\Local\Temp\ipykernel_18500\1617156014.py:5: SettingWithCopyWarning: 
A value is trying to b

In [47]:
df_relevant_cols.head(10)

,TotalPremium,TotalClaims,Gender,PostalCode,Province,VehicleType,RegistrationYear,NewVehicle
0,21.929825,0.0,2,261,2,4,13,1
1,21.929825,0.0,2,261,2,4,13,1
2,0.000000,0.0,2,261,2,4,13,1
3,512.848070,0.0,2,261,2,4,13,1
4,0.000000,0.0,2,261,2,4,13,1
5,3.256435,0.0,2,261,2,4,13,1
6,50.474737,0.0,2,261,2,4,13,1
7,35.332316,0.0,2,261,2,4,13,1
8,0.000000,0.0,2,261,2,4,13,1
9,1.009474,0.0,2,261,2,4,13,1


Split data into features and target variable

In [48]:

X = df_relevant_cols.drop(['TotalClaims', 'TotalPremium'], axis=1)
y = df_relevant_cols['TotalClaims']  # we are predicting TotalClaims


In [69]:
df_relevant_cols['ClaimRatio'] = df_relevant_cols['TotalClaims'] / df_relevant_cols['TotalPremium']


C:\Users\Firew Ayele\AppData\Local\Temp\ipykernel_18500\2236652744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant_cols['ClaimRatio'] = df_relevant_cols['TotalClaims'] / df_relevant_cols['TotalPremium']


In [68]:
print(df_relevant_cols['ClaimRatio'])

0          0.0
1          0.0
2          NaN
3          0.0
4          NaN
          ... 
1000093    0.0
1000094    0.0
1000095    0.0
1000096    0.0
1000097    0.0
Name: ClaimRatio, Length: 1000098, dtype: float64


In [49]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Build Models

Linear Regression

In [50]:
# Initialize and train the model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predictions
y_pred_lr = lr_model.predict(X_test)

Decision Tree

In [62]:
# Initialize and train the model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)

# Predictions
y_pred_dt = dt_model.predict(X_test)

In [63]:
# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

In [64]:
from xgboost import XGBRegressor
# XGBoost
xgb_model = XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

Model Evaluation

In [71]:
# Function to evaluate models
def evaluate_model(y_test, y_pred):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y_test, y_pred)
    return mae, mse, rmse, r2

In [74]:
# Evaluate each model
metrics_lr = evaluate_model(y_test, y_pred_lr)
metrics_dt = evaluate_model(y_test, y_pred_dt)
metrics_rf = evaluate_model(y_test, y_pred_rf)

print(f'Linear Regression: MAE={metrics_lr[0]}, MSE={metrics_lr[1]}, RMSE={metrics_lr[2]}, R²={metrics_lr[3]}')
print(f'Decision Tree: MAE={metrics_dt[0]}, MSE={metrics_dt[1]}, RMSE={metrics_dt[2]}, R²={metrics_dt[3]}')
print(f'Random Forest: MAE={metrics_rf[0]}, MSE={metrics_rf[1]}, RMSE={metrics_rf[2]}, R²={metrics_rf[3]}')


Linear Regression: MAE=129.7813507781326, MSE=4886080.188676738, RMSE=2210.447961087693, R²=6.280666758573528e-05
Decision Tree: MAE=129.06420618037762, MSE=4907972.824057604, RMSE=2215.3945075443344, R²=-0.004417525118234122
Random Forest: MAE=128.6475483446711, MSE=4908052.174972609, RMSE=2215.412416452659, R²=-0.00443376429732556


Feature Importance Analysis

In [75]:
# def evaluate_model(y_test, y_pred, model_name):
#     mse = mean_squared_error(y_test, y_pred)
#     r2 = r2_score(y_test, y_pred)
#     print(f"{model_name} - Mean Squared Error: {mse:.2f}, R2 Score: {r2:.2f}")

# evaluate_model(y_test, y_pred_lr, "Linear Regression")
# evaluate_model(y_test, y_pred_rf, "Random Forest")
# evaluate_model(y_test, y_pred_xgb, "XGBoost")

In [76]:
# # Step 5: Feature Importance with SHAP (for Random Forest)
# explainer = shap.TreeExplainer(rf_model)
# shap_values = explainer.shap_values(X_test)

# # SHAP Summary Plot
# shap.summary_plot(shap_values, X_test)

# # SHAP Dependence Plot (for specific feature)
# # shap.dependence_plot("ClaimRatio", shap_values, X_test)